## Group #3


1. Get the 15 variables from this raster for all Peru departments polygons. This is the link where shapefiles are located. This is the link of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.

2. Then you are going to generate choropleth map using folium for these 15 variables.

3. Save your html in the same folder of your JN. Name your HTML as your branch. This HTML should have all these layers. Please do not forget to use Layer Control.
